[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Jonarod/Bootcamp-Inteligencia-Artificial-Women-Who-Code-Medellin/blob/master/semana%205/ChatBot%20Simple%20Espanol/Chatbot_simple_en_espanol.ipynb)

In [ ]:
import tensorflow as tf
import random
import json
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
# nltk.download('punkt')
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem.snowball import SnowballStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
stemmer = SnowballStemmer("spanish")
toktok = ToktokTokenizer()

def tokenize(sentence):
    """
    Separamos frases en array de palabras/tokens
    """
    return toktok.tokenize(sentence)


def stem(word):
    """
    stemming = encintrar la raiz de una palabra
    ejemplo:
    palabras = ["organisas", "arganisemos", "organisando"]
    -> ["organisar", "organisar", "organisar"]
    """
    return stemmer.stem(word.lower())


def bag_of_words(tokenized_sentence, words):
    """    
    ponemos 1 para cada palabras que existent en la frase, sino ponemos 0
    ejemplo:
    frase = ["hello", "how", "are", "you"]
    palabras = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bag      = [  0 ,    1 ,    0 ,   1 ,    0 ,    0 ,      0]
    """
    # stemming
    sentence_words = [stem(word) for word in tokenized_sentence]
    # iniciemos el bag con 0 para cada palabra
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1

    return bag

In [ ]:
# Importar intents.json
from google.colab import files
files.upload()

Saving intents.json to intents.json


{'intents.json': b'{\n  "intents": [\n    {\n      "tag": "saludo",\n      "patterns": [\n        "Hola",\n        "C\xc3\xb3mo est\xc3\xa1s",\n        "C\xc3\xb3mo est\xc3\xa1s t\xc3\xba",\n        "Qu\xc3\xa9 tal",\n        "Buenos d\xc3\xadas"\n      ],\n      "responses": [\n        "Hola :-)",\n        "Hola, gracias por visitarnos",\n        "Hola, \xc2\xbfqu\xc3\xa9 puedo hacer por ti?",\n        "Hola, \xc2\xbfc\xc3\xb3mo puedo ayudar?"\n      ]\n    },\n    {\n      "tag": "adi\xc3\xb3s",\n      "patterns": ["adi\xc3\xb3s", "hasta luego"],\n      "responses": [\n        "Nos vemos pronto",\n        "Hasta luego, gracias por visitarnos",\n        "Que tenga un lindo d\xc3\xada",\n        "\xc2\xa1Chau! Vuelve pronto."\n      ]\n    },\n    {\n      "tag": "Gracias",\n      "patterns": ["Gracias", "mucho gracias", "eso es \xc3\xbatil"],\n      "responses": ["Encantado de ayudar!", "\xc2\xa1En cualquier momento!", "Mi placer"]\n    },\n    {\n      "tag": "art\xc3\xadculos",\n   

In [ ]:
with open('/content/intents.json', 'r') as f:
    intents = json.load(f)

In [ ]:
all_words = []
tags = []
xy = []

# por cada frase en los intents
for intent in intents['intents']:
    tag = intent['tag']
    # agregar el tag a la lista de tags
    tags.append(tag)

    # para cada padron
    for pattern in intent['patterns']:
        # tokenizacion de cada palabra en la frase
        w = tokenize(pattern)
        # agregamos la palabra al vocabulario
        all_words.extend(w)
        # agregamos los tokens y el tag a xy
        xy.append((w, tag))

# stopwords
ignore_words = stopwords.words('spanish') + ['?', '.', '!',"¿","¡"]
all_words = [stem(w) for w in all_words if w not in ignore_words]
# sacamos los duplicados
all_words = sorted(set(all_words))
tags = sorted(set(tags))


# creacion del juego de entrenamiento
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

In [ ]:
# Hyper-parametros
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = 7


# Modelo
model = tf.keras.models.Sequential([
  tf.keras.Input(shape = input_size),
  tf.keras.layers.Dense(hidden_size, activation='relu'),
  tf.keras.layers.Dense(hidden_size, activation='relu'),
  tf.keras.layers.Dense(output_size, activation='softmax')
])

# compilacion del modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entrenamiento
model.fit(X_train, y_train, epochs=num_epochs)

# Resumen del modelo
model.summary()

# Salvamos el modelo
model.save("/content/my_model")

Epoch 1/1000
1/1 [==============================] - 0s 421ms/step - loss: 1.9481 - accuracy: 0.0870
Epoch 2/1000
1/1 [==============================] - 0s 9ms/step - loss: 1.9452 - accuracy: 0.0870
Epoch 3/1000
1/1 [==============================] - 0s 10ms/step - loss: 1.9423 - accuracy: 0.0870
Epoch 4/1000
1/1 [==============================] - 0s 8ms/step - loss: 1.9395 - accuracy: 0.0870
Epoch 5/1000
1/1 [==============================] - 0s 17ms/step - loss: 1.9367 - accuracy: 0.0870
Epoch 6/1000
1/1 [==============================] - 0s 14ms/step - loss: 1.9340 - accuracy: 0.0870
Epoch 7/1000
1/1 [==============================] - 0s 13ms/step - loss: 1.9313 - accuracy: 0.0870
Epoch 8/1000
1/1 [==============================] - 0s 10ms/step - loss: 1.9288 - accuracy: 0.0870
Epoch 9/1000
1/1 [==============================] - 0s 11ms/step - loss: 1.9262 - accuracy: 0.0870
Epoch 10/1000
1/1 [==============================] - 0s 13ms/step - loss: 1.9237 - accuracy: 0.0870
Epoch 11/1

In [ ]:
!ls

intents.json  my_model	sample_data


In [ ]:
# Importamos el modelo (Deberia hacerse desde otra aplicacion... pero solo para ilustrar)
model = tf.keras.models.load_model('/content/my_model')


bot_name = "Robot"
print("¡Charlemos!")
while True:
    
    sentence = input("tu: ")
    if sentence == "salir":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = tf.convert_to_tensor(X)
    output = model(X)
    prob = output.numpy()
    pred = np.argmax(prob)

     
    tag = tags[pred]
    
    if prob[0][pred] > 0.60:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: Yo no comprendo...")

¡Charlemos!
Robot: Hola, ¿cómo puedo ayudar?
Robot: Aceptamos VISA, Mastercard y Paypal
Robot: Hola :-)
Robot: El envío tarda de 2 a 4 días
Robot: Jaimito, ¿qué planeta va después de Marte? Miércole!
Robot: Yo no comprendo...
Robot: Hasta luego, gracias por visitarnos
